In [91]:
import requests
from bs4 import BeautifulSoup
import json
import time
import csv
import re

In [7]:
url = 'https://www.imdb.com/chart/boxoffice'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [20]:
soup.find('div', {'id': 'boxoffice'}).h4.text

'Weekend of May 10 - 12, 2019'

In [17]:
soup.find('div', {'id': 'boxoffice'}).table.findAll('tr')[1:][0]

<tr>
<td class="posterColumn">
<a href="/title/tt4154796"> <img alt="Avengers: Endgame" height="67" src="https://m.media-amazon.com/images/M/MV5BMTc5MDE2ODcwNV5BMl5BanBnXkFtZTgwMzI2NzQ2NzM@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45"/>
</a> <td class="titleColumn">
<a href="/title/tt4154796" title="Anthony Russo (dir.), Robert Downey Jr., Chris Evans">Avengers: Endgame</a>
</td>
<td class="ratingColumn">
                            $63.3M
                    </td>
<td class="ratingColumn">
<span class="secondaryInfo">$723.7M</span>
</td>
<td class="weeksColumn">3</td>
<td class="watchlistColumn"><div class="wlb_ribbon" data-tconst="tt4154796"></div></td>
</td></tr>

# 미국

In [65]:
start = time.time()
base_imdb_url = 'https://www.imdb.com'
html = requests.get(base_imdb_url + '/chart/boxoffice').text
soup = BeautifulSoup(html, 'html.parser')

boxoffice = {}
boxoffice['date'] = soup.find('div', {'id': 'boxoffice'}).h4.text
boxoffice['movies'] = []

movie_list = []
for movie in soup.find('div', {'id': 'boxoffice'}).table.findAll('tr')[1:]:
    movie_list.append(movie.find('td', class_='titleColumn').a.text)

fieldnames = ['Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Actors', 'Plot',
              'Poster', 'imdbID', 'Production']
for idx, title in enumerate(movie_list):
    temp = {'Country': 'us', 'Rank': str(idx+1), 'Title': title}
    url = f'http://www.omdbapi.com/?apikey=556ed6ed&t={title}&y=2019&plot=full'
    data = json.loads(requests.get(url).text)
    if data['Response'] == 'True':
        for field in fieldnames:
            temp[field] = data[field]
    
        imdbID = temp['imdbID']
        html = requests.get(base_imdb_url + f'/title/{imdbID}').text
        soup = BeautifulSoup(html, 'html.parser')
        temp['imdbScore'] = soup.find('div', 'ratingValue').strong.span.text
        temp['Trailer'] = base_imdb_url + soup.find('div', class_='slate').find('a')['href']
        temp['Trailer_img'] = soup.find('div', class_='slate').find('img')['src']
    
    boxoffice['movies'].append(temp)

print(time.time()-start)

with open('us.csv', 'w') as f:
    fieldnames = ['Country', 'Rank', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
                  'Actors', 'Plot', 'Poster', 'imdbID', 'Production', 'imdbScore', 'Trailer', 'Trailer_img']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for movie in boxoffice['movies']:
        writer.writerow(movie)

18.291301012039185


In [45]:
boxoffice

{'date': 'Weekend of May 10 - 12, 2019',
 'movies': [{'Rank': '1',
   'Title': 'Avengers: Endgame',
   'Year': '2019',
   'Rated': 'PG-13',
   'Released': '26 Apr 2019',
   'Runtime': '181 min',
   'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
   'Director': 'Anthony Russo, Joe Russo',
   'Actors': 'Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth',
   'Plot': "After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to undo Thanos' actions and restore order to the universe.",
   'Poster': 'https://m.media-amazon.com/images/M/MV5BMTc5MDE2ODcwNV5BMl5BanBnXkFtZTgwMzI2NzQ2NzM@._V1_SX300.jpg',
   'imdbID': 'tt4154796',
   'Production': 'Marvel Studios',
   'Trailer': 'https://www.imdb.com/video/imdb/vi2163260441?playlistId=tt4154796',
   'Trailer_img': 'https://m.media-amazon.com/images/M/MV5BOThlNjdhZmQtNzlkOS00M2VjLWI0ZjUtZDExZDI1MjRhZGFkXkEyXkFqcGdeQW1yb3NzZXI@._V1_U

# 호주

In [81]:
base_imdb_url = 'https://www.imdb.com'
mpdaa_url = 'http://www.mpdaa.org.au/customers/mpdaa/mpdaa.nsf/homepage?openform'
html = requests.get(mpdaa_url).text
soup = BeautifulSoup(html, 'html.parser')

boxoffice = {}
boxoffice['movies'] = []

movie_list = []
for td in soup.find('table', class_='BoxOffice').findAll('td', class_='title'):
    movie_list.append(td.text)
    
fieldnames = ['Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Actors', 'Plot',
              'Poster', 'imdbID', 'Production']
for idx, title in enumerate(movie_list):
    temp = {'Country': 'au', 'Rank': str(idx+1), 'Title': title}
    
    year = 2019
    while year > 2016:
        api_url = f'http://www.omdbapi.com/?apikey=556ed6ed&t={title}&y={year}&plot=full'
        data = json.loads(requests.get(api_url).text)
        if data['Response'] == 'True':
            break
        year -= 1
        
    if data['Response'] == 'True':
        temp['Title'] = data['Title']
        for field in fieldnames:
            temp[field] = data[field]
        
        imdbID = temp['imdbID']
        html = requests.get(base_imdb_url + f'/title/{imdbID}').text
        soup = BeautifulSoup(html, 'html.parser')
        temp['imdbScore'] = soup.find('div', 'ratingValue').strong.span.text
        if soup.find('div', class_='slate'):
            trailer = base_imdb_url + soup.find('div', class_='slate').find('a')['href']
            trailer_img = soup.find('div', class_='slate').find('img')['src']
        else:
            trailer = 'N/A'
            trailer_img = 'N/A'
        temp['Trailer'] = trailer
        temp['Trailer_img'] = trailer_img
    else:
        null_filednames = fieldnames + ['imdbScore', 'Trailer', 'Trailer_img']
        for field in null_filednames:
            temp[field] = 'N/A'
            
    boxoffice['movies'].append(temp)

In [82]:
with open('au.csv', 'w') as f:
    fieldnames = ['Country', 'Rank', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
                  'Actors', 'Plot', 'Poster', 'imdbID', 'Production', 'imdbScore', 'Trailer', 'Trailer_img']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for movie in boxoffice['movies']:
        writer.writerow(movie)

# Netflix

In [133]:
url = 'https://www.techjunkie.com/best-netflix-original-shows/'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
shows = []
for idx, show in enumerate(soup.findAll('div', class_='tj-best-list__item')[-20:][::-1]):
    temp = {'Rank': idx+1}
    title_words = show.find('span', class_='tj-best-list-featured-item__title').text.split()[1:]
    title = ' '.join(title_words)
    temp['Title'] = title
    netflix_url = show.find('a', class_='button')['href']
    temp['Netflix_url'] = netflix_url
    netflix_html = requests.get(netflix_url).text
    netflix_soup = BeautifulSoup(netflix_html, 'html.parser')
    title_info = netflix_soup.find('div', class_='title-info')
    temp['Year'] = title_info.find('span', class_='item-year').text
    temp['Rated'] = title_info.find('span', class_='maturity-number').text.strip()
    temp['Season'] = title_info.find('span', class_='item-runtime').text
    temp['Genre'] = title_info.find('span', class_='item-genre').text
    temp['Plot'] = title_info.find('div', {'data-uia': 'title-info-synopsis'}).text
    actors = title_info.find('span', {'data-uia': 'info-starring'})
    if actors:
        actors = actors.text
    else:
        actors = 'N/A'
    temp['Actors'] = actors
    temp['Trailer'] = 'N/A'
    temp['Thumbnail'] = netflix_soup.findAll('div', class_='season')[-1].figure.img['src']
    shows.append(temp)

In [134]:
shows

[{'Rank': 1,
  'Title': 'Lucifer',
  'Netflix_url': 'https://www.netflix.com/title/80057918',
  'Year': '2016',
  'Rated': '18',
  'Season': '4 Seasons',
  'Genre': 'TV Sci-Fi & Fantasy',
  'Plot': 'Bored with being the Lord of Hell, the devil relocates to Los Angeles, where he opens a nightclub and forms a connection with a homicide detective.',
  'Actors': 'Tom Ellis,\xa0Lauren German,\xa0Kevin Alejandro',
  'Trailer': 'N/A',
  'Thumbnail': 'https://occ-0-1360-1009.1.nflxso.net/art/572a7/38d4ffd82996ca1c90f93bee8c5becb4618572a7.jpg'},
 {'Rank': 2,
  'Title': 'I Think You Should Leave with Tim Robinson',
  'Netflix_url': 'https://www.netflix.com/title/80986854',
  'Year': '2019',
  'Rated': '18',
  'Season': '1 Season',
  'Genre': 'US TV Shows',
  'Plot': 'There is no such thing as an ordinary interaction in this offbeat sketch comedy series that features a deep roster of guest stars.',
  'Actors': 'Tim Robinson,\xa0Andy Samberg,\xa0Sam Richardson',
  'Trailer': 'N/A',
  'Thumbnail': 

In [135]:
with open('netflix.csv', 'w') as f:
    fieldnames = ['Rank', 'Title', 'Netflix_url', 'Year', 'Rated', 'Season', 'Genre', 'Plot', 'Actors',
                  'Trailer', 'Thumbnail']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for show in shows:
        writer.writerow(show)